# Supervised Fine-tuning (SFT) with TRL

Fine-tuning requires a GPU. If you don't have one locally, you can run this notebook for free on [Google Colab](https://colab.research.google.com/github/Liquid4All/cookbook/blob/main/finetuning/notebooks/sft_with_trl.ipynb) using a free NVIDIA T4 GPU instance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Liquid4All/cookbook/blob/main/finetuning/notebooks/sft_with_trl.ipynb)

### What's in this notebook?

In this notebook you will learn how to perform Supervised Fine-tuning (SFT) using the Hugging Face TRL (Transformer Reinforcement Learning) library.
We will use the [LFM2.5-1.2B-Instruct](https://docs.liquid.ai/docs/models/lfm25-1.2b-instruct) model and fine-tune it on the SmolTalk dataset. You'll learn both standard SFT and parameter-efficient fine-tuning with LoRA for constrained hardware.

We will cover
- Environment setup
- Data preparation
- Model training with TRL's SFTTrainer
- LoRA for parameter-efficient fine-tuning
- Local inference with your new model
- Model saving and exporting it into the format you need for **deployment**.

### Deployment options

LFM2.5 models are small and efficient, enabling deployment across a wide range of platforms:

<table align="left">
  <tr>
    <th>Deployment Target</th>
    <th>Use Case</th>
  </tr>
  <tr>
    <td>📱 <a href="https://docs.liquid.ai/leap/edge-sdk/android/android-quick-start-guide"><b>Android</b></a></td>
    <td>Mobile apps on Android devices</td>
  </tr>
  <tr>
    <td>📱 <a href="https://docs.liquid.ai/leap/edge-sdk/ios/ios-quick-start-guide"><b>iOS</b></a></td>
    <td>Mobile apps on iPhone/iPad</td>
  </tr>
  <tr>
    <td>🍎 <a href="https://docs.liquid.ai/docs/inference/mlx"><b>Apple Silicon Mac</b></a></td>
    <td>Local inference on Mac with MLX</td>
  </tr>
  <tr>
    <td>🦙 <a href="https://docs.liquid.ai/docs/inference/llama-cpp"><b>llama.cpp</b></a></td>
    <td>Local deployments on any hardware</td>
  </tr>
  <tr>
    <td>🦙 <a href="https://docs.liquid.ai/docs/inference/ollama"><b>Ollama</b></a></td>
    <td>Local inference with easy setup</td>
  </tr>
  <tr>
    <td>🖥️ <a href="https://docs.liquid.ai/docs/inference/lm-studio"><b>LM Studio</b></a></td>
    <td>Desktop app for local inference</td>
  </tr>
  <tr>
    <td>⚡ <a href="https://docs.liquid.ai/docs/inference/vllm"><b>vLLM</b></a></td>
    <td>Cloud deployments with high throughput</td>
  </tr>
  <tr>
    <td>☁️ <a href="https://docs.liquid.ai/docs/inference/modal-deployment"><b>Modal</b></a></td>
    <td>Serverless cloud deployment</td>
  </tr>
  <tr>
    <td>🏗️ <a href="https://docs.liquid.ai/docs/inference/baseten-deployment"><b>Baseten</b></a></td>
    <td>Production ML infrastructure</td>
  </tr>
  <tr>
    <td>🚀 <a href="https://docs.liquid.ai/docs/inference/fal-deployment"><b>Fal</b></a></td>
    <td>Fast inference API</td>
  </tr>
</table>

### Need help building with our models and tools?
Join the Liquid AI Discord Community and ask.

<a href="https://discord.com/invite/liquid-ai"><img src="https://img.shields.io/discord/1385439864920739850?color=7289da&label=Join%20Discord&logo=discord&logoColor=white" alt="Join Discord"></a>

And now, let the fine tune begin!

# 📦 Installation & Setup

First, let's install all the required packages:


In [ ]:
!pip install transformers==4.54.0 trl>=0.18.2 peft>=0.15.2

Let's now verify the packages are installed correctly

In [ ]:
import torch
import transformers
import trl
import os
os.environ["WANDB_DISABLED"] = "true"

print(f"📦 PyTorch version: {torch.__version__}")
print(f"🤗 Transformers version: {transformers.__version__}")
print(f"📊 TRL version: {trl.__version__}")

# Loading the model from Transformers 🤗



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, HTML, Markdown
import torch

# Select a model to fine-tune from the list
lfm_models = [
    "LiquidAI/LFM2.5-1.2B-Instruct",
    "LiquidAI/LFM2.5-1.2B-JP",
    "LiquidAI/LFM2-8B-A1B",
    "LiquidAI/LFM2-2.6B-Exp",
    "LiquidAI/LFM2-2.6B",
    "LiquidAI/LFM2-700M",
    "LiquidAI/LFM2-350M",
]

# Model to fine-tune
model_id = "LiquidAI/LFM2.5-1.2B-Instruct"

print("📚 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

print("🧠 Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="bfloat16",
#   attn_implementation="flash_attention_2" <- uncomment on compatible GPU
)

print("✅ Local model loaded successfully!")
print(f"🔢 Parameters: {model.num_parameters():,}")
print(f"📖 Vocab size: {len(tokenizer)}")
print(f"💾 Model size: ~{model.num_parameters() * 2 / 1e9:.1f} GB (bfloat16)")

# 🎯 Part 1: Supervised Fine-Tuning (SFT)

SFT teaches the model to follow instructions by training on input-output pairs (instruction vs response). This is the foundation for creating instruction-following models.

## Load an SFT Dataset

We will use [HuggingFaceTB/smoltalk](https://huggingface.co/datasets/HuggingFaceTB/smoltalk), limiting ourselves to the first 5k samples for brevity. Feel free to change the limit by changing the slicing index in the parameter `split`.

In [ ]:
from datasets import load_dataset

print("📥 Loading SFT dataset...")
train_dataset_sft = load_dataset("HuggingFaceTB/smoltalk", "all", split="train[:5000]")
eval_dataset_sft = load_dataset("HuggingFaceTB/smoltalk", "all", split="test[:500]")

print("✅ SFT Dataset loaded:")
print(f"   📚 Train samples: {len(train_dataset_sft)}")
print(f"   🧪 Eval samples: {len(eval_dataset_sft)}")
print(f"\n📝 Single Sample: {train_dataset_sft[0]['messages']}")

## Launch Training

We are now ready to launch an SFT run with `SFTTrainer`, feel free to modify `SFTConfig` to play around with different configurations.



In [ ]:
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(
    output_dir="./lfm2-sft",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    warmup_steps=100,
    warmup_ratio=0.2,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    report_to=None,
    bf16=False # <- not all colab GPUs support bf16
)

print("🏗️  Creating SFT trainer...")
sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset_sft,
    eval_dataset=eval_dataset_sft,
    processing_class=tokenizer,
)

print("\n🚀 Starting SFT training...")
sft_trainer.train()

print("🎉 SFT training completed!")

sft_trainer.save_model()
print(f"💾 SFT model saved to: {sft_config.output_dir}")

# 🎛️ Part 2: LoRA + SFT (Parameter-Efficient Fine-tuning)

LoRA (Low-Rank Adaptation) allows efficient fine-tuning by only training a small number of additional parameters. Perfect for limited compute resources!


## Wrap the model with PEFT

We specify target modules that will be finetuned while the rest of the models weights remains frozen. Feel free to modify the `r` (rank) value:
- higher -> better approximation of full-finetuning
- lower -> needs even less compute resources

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

GLU_MODULES = ["w1", "w2", "w3"]
MHA_MODULES = ["q_proj", "k_proj", "v_proj", "out_proj"]
CONV_MODULES = ["in_proj", "out_proj"]

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,  # <- lower values = fewer parameters
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=GLU_MODULES + MHA_MODULES + CONV_MODULES,
    bias="none",
    modules_to_save=None,
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

print("✅ LoRA configuration applied!")
print(f"🎛️  LoRA rank: {lora_config.r}")
print(f"📊 LoRA alpha: {lora_config.lora_alpha}")
print(f"🎯 Target modules: {lora_config.target_modules}")

## Launch Training

Now ready to launch the SFT training, but this time with the LoRA-wrapped model

In [ ]:
from trl import SFTConfig, SFTTrainer

lora_sft_config = SFTConfig(
    output_dir="./lfm2-sft-lora",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    warmup_steps=100,
    warmup_ratio=0.2,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    report_to=None,
)

print("🏗️  Creating LoRA SFT trainer...")
lora_sft_trainer = SFTTrainer(
    model=lora_model,
    args=lora_sft_config,
    train_dataset=train_dataset_sft,
    eval_dataset=eval_dataset_sft,
    processing_class=tokenizer,
)

print("\n🚀 Starting LoRA + SFT training...")
lora_sft_trainer.train()

print("🎉 LoRA + SFT training completed!")

lora_sft_trainer.save_model()
print(f"💾 LoRA model saved to: {lora_sft_config.output_dir}")

## Save merged model

Merge the extra weights learned with LoRA back into the model to obtain a "normal" model checkpoint.

In [ ]:
print("\n🔄 Merging LoRA weights...")
merged_model = lora_model.merge_and_unload()
merged_model.save_pretrained("./lfm2-lora-merged")
tokenizer.save_pretrained("./lfm2-lora-merged")
print("💾 Merged model saved to: ./lfm2-lora-merged")